SETUP SQL DATABASE


In [ ]:
import sqlite3
import pandas as pd

# Create in-memory SQL database
conn = sqlite3.connect(":memory:")
cur = conn.cursor()

print("Database Created Successfully")


Database Created Successfully


CREATE TABLES

In [ ]:
cur.executescript("""
CREATE TABLE assets(
    asset_id INTEGER PRIMARY KEY AUTOINCREMENT,
    hostname TEXT,
    ip_address TEXT,
    department TEXT
);

CREATE TABLE users(
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT,
    role TEXT
);

CREATE TABLE threat_intel(
    intel_id INTEGER PRIMARY KEY AUTOINCREMENT,
    ip TEXT,
    threat_type TEXT,
    severity INTEGER
);

CREATE TABLE event_logs(
    log_id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    src_ip TEXT,
    dest_ip TEXT,
    bytes INTEGER,
    protocol TEXT,
    threat_score REAL,
    user_id INTEGER,
    asset_id INTEGER
);
""")

print("Tables Created Successfully!")


Tables Created Successfully!


INSERT SIMPLE & CLEAN DATA

In [ ]:
cur.executescript("""
INSERT INTO assets(hostname, ip_address, department) VALUES
('Server01','10.0.0.10','Finance'),
('Server02','10.0.0.11','HR');

INSERT INTO users(username, role) VALUES
('admin','Security'),
('john','Developer');

INSERT INTO threat_intel(ip, threat_type, severity) VALUES
('45.90.12.1','Malware C2',9),
('172.198.5.22','Ransomware',10);

INSERT INTO event_logs(timestamp, src_ip, dest_ip, bytes, protocol, threat_score, user_id, asset_id)
VALUES
('2025-02-01 10:00','45.90.12.1','10.0.0.10',5000,'TCP',8.5,1,1),
('2025-02-01 10:20','192.168.1.5','10.0.0.11',250,'UDP',2.0,2,2),
('2025-02-01 10:40','172.198.5.22','10.0.0.11',12000,'TCP',9.5,1,2);
""")

print("Sample Data Inserted!")


Sample Data Inserted!


Using pandas to display ALL SQL RESULTS nicely

This helper function prints tables in clean format.

In [ ]:
def show(query):
    return pd.read_sql_query(query, conn)


SHOW TABLE STRUCTURE (TABULAR OUTPUT)
Assets Structure

In [ ]:
show("PRAGMA table_info(assets)")


,cid,name,type,notnull,dflt_value,pk
0,0,asset_id,INTEGER,0,None,1
1,1,hostname,TEXT,0,None,0
2,2,ip_address,TEXT,0,None,0
3,3,department,TEXT,0,None,0


Users Structure

In [ ]:
show("PRAGMA table_info(users)")


,cid,name,type,notnull,dflt_value,pk
0,0,user_id,INTEGER,0,None,1
1,1,username,TEXT,0,None,0
2,2,role,TEXT,0,None,0


Threat Intel Structure

In [ ]:
show("PRAGMA table_info(threat_intel)")


,cid,name,type,notnull,dflt_value,pk
0,0,intel_id,INTEGER,0,None,1
1,1,ip,TEXT,0,None,0
2,2,threat_type,TEXT,0,None,0
3,3,severity,INTEGER,0,None,0


Event Logs Structure

In [ ]:
show("PRAGMA table_info(event_logs)")


,cid,name,type,notnull,dflt_value,pk
0,0,log_id,INTEGER,0,None,1
1,1,timestamp,TEXT,0,None,0
2,2,src_ip,TEXT,0,None,0
3,3,dest_ip,TEXT,0,None,0
4,4,bytes,INTEGER,0,None,0
5,5,protocol,TEXT,0,None,0
6,6,threat_score,REAL,0,None,0
7,7,user_id,INTEGER,0,None,0
8,8,asset_id,INTEGER,0,None,0


VIEW ALL LOGS

In [ ]:
show("SELECT * FROM event_logs")


,log_id,timestamp,src_ip,dest_ip,bytes,protocol,threat_score,user_id,asset_id
0,1,2025-02-01 10:00,45.90.12.1,10.0.0.10,5000,TCP,8.5,1,1
1,2,2025-02-01 10:20,192.168.1.5,10.0.0.11,250,UDP,2.0,2,2
2,3,2025-02-01 10:40,172.198.5.22,10.0.0.11,12000,TCP,9.5,1,2


Detect Malicious IPs (Easy Advanced JOIN)

In [ ]:
show("""
SELECT e.timestamp, e.src_ip, e.dest_ip, t.threat_type, t.severity
FROM event_logs e
JOIN threat_intel t
ON e.src_ip = t.ip
""")


,timestamp,src_ip,dest_ip,threat_type,severity
0,2025-02-01 10:00,45.90.12.1,10.0.0.10,Malware C2,9
1,2025-02-01 10:40,172.198.5.22,10.0.0.11,Ransomware,10


Find Most Attacked Asset

In [ ]:
show("""
SELECT a.hostname, COUNT(e.log_id) AS attack_count
FROM assets a
JOIN event_logs e ON a.asset_id = e.asset_id
GROUP BY a.hostname
ORDER BY attack_count DESC
""")


,hostname,attack_count
0,Server02,2
1,Server01,1


High Threat Score Events Only

In [ ]:
show("SELECT * FROM event_logs WHERE threat_score >= 8")


,log_id,timestamp,src_ip,dest_ip,bytes,protocol,threat_score,user_id,asset_id
0,1,2025-02-01 10:00,45.90.12.1,10.0.0.10,5000,TCP,8.5,1,1
1,3,2025-02-01 10:40,172.198.5.22,10.0.0.11,12000,TCP,9.5,1,2


Total Traffic Per Source IP

In [ ]:
show("SELECT src_ip, SUM(bytes) AS total_bytes FROM event_logs GROUP BY src_ip")


,src_ip,total_bytes
0,172.198.5.22,12000
1,192.168.1.5,250
2,45.90.12.1,5000


Average Threat Score Per Asset

In [ ]:
show("""
SELECT a.hostname, AVG(e.threat_score) AS avg_risk
FROM assets a
JOIN event_logs e ON a.asset_id = e.asset_id
GROUP BY a.hostname
""")


,hostname,avg_risk
0,Server01,8.50
1,Server02,5.75


Alert System (CASE Statement)

In [ ]:
show("""
SELECT log_id, src_ip,
CASE
    WHEN threat_score >= 9 THEN 'CRITICAL'
    WHEN threat_score >= 7 THEN 'HIGH'
    ELSE 'LOW'
END AS alert_level
FROM event_logs
""")


,log_id,src_ip,alert_level
0,1,45.90.12.1,HIGH
1,2,192.168.1.5,LOW
2,3,172.198.5.22,CRITICAL


Add New Column Easily

In [ ]:
cur.execute("ALTER TABLE event_logs ADD COLUMN reviewed TEXT")
conn.commit()

show("PRAGMA table_info(event_logs)")


,cid,name,type,notnull,dflt_value,pk
0,0,log_id,INTEGER,0,None,1
1,1,timestamp,TEXT,0,None,0
2,2,src_ip,TEXT,0,None,0
3,3,dest_ip,TEXT,0,None,0
4,4,bytes,INTEGER,0,None,0
5,5,protocol,TEXT,0,None,0
6,6,threat_score,REAL,0,None,0
7,7,user_id,INTEGER,0,None,0
8,8,asset_id,INTEGER,0,None,0
9,9,reviewed,TEXT,0,None,0


Update Column Example

In [ ]:
cur.execute("UPDATE event_logs SET reviewed='No'")
conn.commit()

show("SELECT log_id, reviewed FROM event_logs")


,log_id,reviewed
0,1,No
1,2,No
2,3,No


Top Traffic Destinations

In [ ]:
show("SELECT dest_ip, SUM(bytes) AS total FROM event_logs GROUP BY dest_ip")


,dest_ip,total
0,10.0.0.10,5000
1,10.0.0.11,12250


Events With Above Average Bytes

In [ ]:
show("""
SELECT * FROM event_logs
WHERE bytes > (SELECT AVG(bytes) FROM event_logs)
""")


,log_id,timestamp,src_ip,dest_ip,bytes,protocol,threat_score,user_id,asset_id,reviewed
0,3,2025-02-01 10:40,172.198.5.22,10.0.0.11,12000,TCP,9.5,1,2,None


GROUP BY with SUM()

In [ ]:
show("""
SELECT src_ip, SUM(bytes) AS total_bytes
FROM event_logs
GROUP BY src_ip
ORDER BY total_bytes DESC
""")


,src_ip,total_bytes
0,172.198.5.22,12000
1,45.90.12.1,5000
2,192.168.1.5,250


 GROUP BY with AVG()
Average threat score for each asset


In [ ]:
show("""
SELECT t.threat_type, COUNT(e.log_id) AS attack_count
FROM event_logs e
JOIN threat_intel t ON e.src_ip = t.ip
GROUP BY t.threat_type
ORDER BY attack_count DESC
""")


,threat_type,attack_count
0,Ransomware,1
1,Malware C2,1


GROUP BY with COUNT()
Number of attacks per threat type


In [ ]:
show("""
SELECT t.threat_type, COUNT(e.log_id) AS attack_count
FROM event_logs e
JOIN threat_intel t ON e.src_ip = t.ip
GROUP BY t.threat_type
ORDER BY attack_count DESC
""")


,threat_type,attack_count
0,Ransomware,1
1,Malware C2,1


GROUP BY with HAVING
Show only IPs whose total traffic exceeds 5000 bytes

In [ ]:
show("""
SELECT src_ip, SUM(bytes) AS total_bytes
FROM event_logs
GROUP BY src_ip
HAVING total_bytes > 5000
ORDER BY total_bytes DESC
""")


,src_ip,total_bytes
0,172.198.5.22,12000


GROUP BY Multiple Columns
Bytes transferred grouped by protocol AND destination
show

In [ ]:
show("""
SELECT protocol, dest_ip, SUM(bytes) AS total_bytes
FROM event_logs
GROUP BY protocol, dest_ip
ORDER BY total_bytes DESC
""")


,protocol,dest_ip,total_bytes
0,TCP,10.0.0.11,12000
1,TCP,10.0.0.10,5000
2,UDP,10.0.0.11,250


INNER JOIN
Show event details along with user name & asset hostname

In [ ]:
show("""
SELECT e.timestamp, u.username, a.hostname, e.src_ip, e.dest_ip, e.threat_score
FROM event_logs e
JOIN users u ON e.user_id = u.user_id
JOIN assets a ON e.asset_id = a.asset_id
ORDER BY e.timestamp
""")


,timestamp,username,hostname,src_ip,dest_ip,threat_score
0,2025-02-01 10:00,admin,Server01,45.90.12.1,10.0.0.10,8.5
1,2025-02-01 10:20,john,Server02,192.168.1.5,10.0.0.11,2.0
2,2025-02-01 10:40,admin,Server02,172.198.5.22,10.0.0.11,9.5


LEFT JOIN
Show all assets and whether they have been attacked

In [ ]:
show("""
SELECT a.hostname, a.ip_address, e.threat_score
FROM assets a
LEFT JOIN event_logs e ON a.asset_id = e.asset_id
ORDER BY a.hostname
""")


,hostname,ip_address,threat_score
0,Server01,10.0.0.10,8.5
1,Server02,10.0.0.11,2.0
2,Server02,10.0.0.11,9.5


JOIN With Threat Intel
Identify events triggered by known malicious IPs

In [ ]:
show("""
SELECT e.timestamp, e.src_ip, t.threat_type, t.severity, e.dest_ip
FROM event_logs e
JOIN threat_intel t ON e.src_ip = t.ip
ORDER BY t.severity DESC
""")


,timestamp,src_ip,threat_type,severity,dest_ip
0,2025-02-01 10:40,172.198.5.22,Ransomware,10,10.0.0.11
1,2025-02-01 10:00,45.90.12.1,Malware C2,9,10.0.0.10


SELF JOIN
Detect same IP communicating with multiple assets

In [ ]:
show("""
SELECT e1.src_ip, e1.dest_ip AS dest1, e2.dest_ip AS dest2
FROM event_logs e1
JOIN event_logs e2
ON e1.src_ip = e2.src_ip AND e1.log_id != e2.log_id
""")


,src_ip,dest1,dest2


JOIN + GROUP BY
Most targeted department by attacks

In [ ]:
show("""
SELECT a.department, COUNT(e.log_id) AS attacks
FROM assets a
JOIN event_logs e ON a.asset_id = e.asset_id
GROUP BY a.department
ORDER BY attacks DESC
""")


,department,attacks
0,HR,2
1,Finance,1


JOIN with CASE (Threat Level Classification)

In [ ]:
show("""
SELECT
    e.src_ip,
    e.dest_ip,
    e.bytes,
    CASE
        WHEN e.threat_score >= 9 THEN 'CRITICAL'
        WHEN e.threat_score >= 7 THEN 'HIGH'
        WHEN e.threat_score >= 4 THEN 'MEDIUM'
        ELSE 'LOW'
    END AS severity_level
FROM event_logs e
ORDER BY threat_score DESC
""")


,src_ip,dest_ip,bytes,severity_level
0,172.198.5.22,10.0.0.11,12000,CRITICAL
1,45.90.12.1,10.0.0.10,5000,HIGH
2,192.168.1.5,10.0.0.11,250,LOW
